* The basic function and constants:

In [29]:
def net_income(visitor):
    if visitor<=basic_visitor:
        major_cost=fixed_cost
    else:
        major_cost=fixed_cost+(visitor-basic_visitor)*cloud_sever_fee
    
    revenue=visitor*ad_sponsorship+visitor*subscription_fee*convert_rate

    net_income=revenue-major_cost
    return net_income

content_cost=7000000
other_cost=300000
infrastructure_cost=50000000
cloud_sever_fee=0.001
subscription_fee=15
convert_rate=0.1
ad_sponsorship=0.8
basic_visitor=50000

fixed_cost=content_cost+other_cost+infrastructure_cost

* a basic method to get to the answer:

In [88]:
%%time

i=0
while net_income(i)<0:
    i=i+1
print('visitor=%s is the break even point'%i)

visitor=24923859 is the break even point
CPU times: user 18.8 s, sys: 103 ms, total: 18.9 s
Wall time: 19.5 s


* An advanced method mentioned by @kaiwenxu94：
    - Firstly plus 10000 by 10000, then minus back 1 by 1

In [27]:
%%time

i=0
while net_income(i)<0:
    i=i+10000
while net_income(i)>=0:
    i=i-1
print('visitor=%s is the break even point'%(i+1))

visitor=24923859 is the break even point
CPU times: user 8.02 ms, sys: 291 µs, total: 8.31 ms
Wall time: 8.29 ms


* Inspired by his intelligence, I come up with a more universal way to figure out the break point. Generally, it is +100000 circularly and then -10000 in the same way until +/-1:

In [39]:
%%time

def sway():
    i=0
    e=len(str(fixed_cost))-1                #estimated scale of addend used firstly
    f=int(e%2!=0)                           #make sure the first addend’s sign is +, since we use a (-1)**n to control +/- later
    while e>=0:                             #end loop until 10**e=1
        while net_income(i)*(-1)**(e+f)<0:  #try to get a number bigger/smaller(depending on how many rounds) than the point
            #print(i)                       #memo of how f works: when e is odd number, f become 1, so the 1st e is always even number
            i=i+10**e*(-1)**(e+f)           #use a (-1)**n to control +/-; use f=int(e%2!=0) to make sure the first addend‘s sign is +
        e=e-1                               #promote precision and plus/minus again 
    if net_income(i)<0:                     #if the loop ends with minus, we should add it back above 0
        i=i+1
        #print(i)
    return i
print('visitor=%s is the break even point'% sway())

visitor=24923859 is the break even point
CPU times: user 392 µs, sys: 103 µs, total: 495 µs
Wall time: 442 µs


* Motivated by Pili's instruction, I found the arithmetic speed can be even faster using bisection method(二分法)！I am not sure whether it is the exact bisection method but it really works:

In [40]:
%%time

def bisection():
    i=0
    e=len(str(fixed_cost))-1        #estimated scale of addend used firstly
    g=10**e
    while net_income(i)<0:          #firstly find a section (i0，i1) containing the break even point
        i=i+g
        #print(i)
    i0=i-g
    i1=i
    while i1-i0>2:                  #avoid the endless loop when i1-i0=2, since there is i1=middle+1 in bisection
        middle=(i0+i1)//2
        if net_income(middle)<0:
            i0=middle               #no need for i0=middle-1 since fractional parts have been rounded down by '//'
        elif net_income(middle)>=0:
            i1=middle+1             #let i1=middle+1 so that if middle<x<middle+1 when net_income(x)=0, the x won't be skipped
        #print(i0,i1)
    if net_income(i1-1)>=0:         #the point could be in (i1-1,i1) or (i1-2,i1-1)
        i1=i1-1
        #print(i1)
    return i1
print('visitor=%s is the break even point'% bisection())

visitor=24923859 is the break even point
CPU times: user 206 µs, sys: 79 µs, total: 285 µs
Wall time: 230 µs


* Of course we can figure out the answer directly, which shows the outputs of sway() and bisection() are correct:

In [24]:
answer=int((fixed_cost-basic_visitor*cloud_sever_fee)/((ad_sponsorship+subscription_fee*convert_rate)-cloud_sever_fee))+1
print(answer)

24923859


* Repeat sway() for 100000 times for a more stable time test:

In [22]:
%%time

for i in range(1,100000):
    sway()

CPU times: user 8.02 s, sys: 34.6 ms, total: 8.06 s
Wall time: 8.2 s


* Repeat bisection() for 100000 times. It shows bisection()‘s time consumption is about 1/3 of sway()’s on this stage:

In [23]:
%%time

for i in range(1,100000):
    bisection()

CPU times: user 2.67 s, sys: 18.3 ms, total: 2.68 s
Wall time: 2.74 s
